### Generate plots

Before running this notebook you have to generate the input dataframe, **df_all.tsv**, with the main.py script.
Optionally, a mapping file specifying group and label of methods can be used to select only one method (the best) for each group and each namespace.

#### Output
* 3 figures, *fig_<metric>_<namespace>.png*
* 1 TSV, *eval_<metric>.tsv*

In [70]:
# Input
df_file = "../../CAFA-evaluator_data/CAFA3/results/df_all.tsv"
out_folder = "../../CAFA-evaluator_data/CAFA3/results"

# Set to None if you don't want to use it. Results will not be grouped
# name_file = None
names_file = "../../CAFA-evaluator_data/CAFA3/names.tsv"

# Select a metric
# metric, cols = ('f', ['rc', 'pr'])
metric, cols =  ('wf', ['wrc', 'wpr'])
# metric, cols = ('s', ['ru', 'mi'])

In [71]:
import pandas as pd
import matplotlib.pyplot as plt
plt.set_loglevel("info")

In [72]:
df = pd.read_csv(df_file, sep="\t")
df

,filename,ns,tau,cov,pr,rc,f,wpr,wrc,wf,mi,ru,s
0,ZhuLab1_GOLabeler-2015.cafa,biological_process,0.01,1.000000,0.020865,0.844478,0.040723,0.014994,0.745858,0.029397,5428.198473,33.907521,5428.304374
1,ZhuLab1_GOLabeler-2015.cafa,biological_process,0.02,1.000000,0.020865,0.844478,0.040723,0.014994,0.745858,0.029397,5428.198473,33.907521,5428.304374
2,ZhuLab1_GOLabeler-2015.cafa,biological_process,0.03,1.000000,0.020865,0.844478,0.040723,0.014994,0.745858,0.029397,5428.198473,33.907521,5428.304374
3,ZhuLab1_GOLabeler-2015.cafa,biological_process,0.04,1.000000,0.020866,0.844478,0.040726,0.014996,0.745858,0.029400,5427.630821,33.907521,5427.736734
4,ZhuLab1_GOLabeler-2015.cafa,biological_process,0.05,1.000000,0.020877,0.844478,0.040747,0.015006,0.745858,0.029419,5424.371807,33.907521,5424.477783
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11989,sysbiogroup.cafa,molecular_function,0.95,0.709534,0.418023,0.073154,0.124518,0.425644,0.097213,0.158277,11.905188,58.458560,59.658501
11990,sysbiogroup.cafa,molecular_function,0.96,0.709534,0.418023,0.073154,0.124518,0.425644,0.097213,0.158277,11.905188,58.458560,59.658501
11991,sysbiogroup.cafa,molecular_function,0.97,0.709534,0.418023,0.073154,0.124518,0.425644,0.097213,0.158277,11.905188,58.458560,59.658501
11992,sysbiogroup.cafa,molecular_function,0.98,0.709534,0.418023,0.073154,0.124518,0.425644,0.097213,0.158277,11.905188,58.458560,59.658501


In [73]:
methods = pd.read_csv(names_file, delim_whitespace=True)
methods

,filename,group,label
0,anacletolab.cafa,anacletolab,anacletolab
1,caolab1.cafa,caolab,caolab1
2,caolab4_1.cafa,caolab,caolab4_1
3,caolab4_2.cafa,caolab,caolab4_2
4,caolab4_3.cafa,caolab,caolab4_3
5,cbrcborg_1.cafa,cbrcborg,cbrcborg_1
6,cbrcborg_2.cafa,cbrcborg,cbrcborg_2
7,cbrcborg_3.cafa,cbrcborg,cbrcborg_3
8,DeepGO_1.cafa,DeepGO,DeepGO_1
9,DeepGO_2.cafa,DeepGO,DeepGO_2


In [74]:
if names_file is None:
    df['group'] = df['filename']
    df['label'] = df['filename']
    df['is_baseline'] = False
else:
    # Set method information (optional)
    df = pd.merge(df, methods, on='filename', how='left')
    df['group'].fillna(df['filename'], inplace=True)
    df['label'].fillna(df['filename'], inplace=True)
    if 'is_baseline' not in df:
        df['is_baseline'] = False
    else:
        df['is_baseline'].fillna(False, inplace=True)
df = df.drop(columns='filename').set_index(['group', 'label', 'ns', 'tau'])
df

cov  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  1.000000   
                                                      0.02  1.000000   
                                                      0.03  1.000000   
                                                      0.04  1.000000   
                                                      0.05  1.000000   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.709534   
                                                      0.96  0.709534   
                                                      0.97  0.709534   
                                                      0.98  0.709534   
                                                      0.99  0.709534   

                                                                  pr  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.020865   
                                                      0.02  0.020865   
                                                      0.03  0.020865   
                                                      0.04  0.020866   
                                                      0.05  0.020877   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.418023   
                                                      0.96  0.418023   
                                                      0.97  0.418023   
                                                      0.98  0.418023   
                                                      0.99  0.418023   

                                                                  rc  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.844478   
                                                      0.02  0.844478   
                                                      0.03  0.844478   
                                                      0.04  0.844478   
                                                      0.05  0.844478   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.073154   
                                                      0.96  0.073154   
                                                      0.97  0.073154   
                                                      0.98  0.073154   
                                                      0.99  0.073154   

                                                                   f  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.040723   
                                                      0.02  0.040723   
                                                      0.03  0.040723   
                                                      0.04  0.040726   
                                                      0.05  0.040747   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.124518   
                                                      0.96  0.124518   
                                                      0.97  0.124518   
                                                      0.98  0.124518   
                                                      0.99  0.124518   

                                                                 wpr  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.014994   
                                                      0.02  0.014994   
                             

In [75]:
# Assign colors based on group
cmap = plt.get_cmap('tab20')
df['colors'] = df.index.get_level_values('group')
df['colors'] = pd.factorize(df['colors'])[0]
df['colors'] = df['colors'].apply(lambda x: cmap.colors[x % len(cmap.colors)])
df

cov  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  1.000000   
                                                      0.02  1.000000   
                                                      0.03  1.000000   
                                                      0.04  1.000000   
                                                      0.05  1.000000   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.709534   
                                                      0.96  0.709534   
                                                      0.97  0.709534   
                                                      0.98  0.709534   
                                                      0.99  0.709534   

                                                                  pr  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.020865   
                                                      0.02  0.020865   
                                                      0.03  0.020865   
                                                      0.04  0.020866   
                                                      0.05  0.020877   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.418023   
                                                      0.96  0.418023   
                                                      0.97  0.418023   
                                                      0.98  0.418023   
                                                      0.99  0.418023   

                                                                  rc  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.844478   
                                                      0.02  0.844478   
                                                      0.03  0.844478   
                                                      0.04  0.844478   
                                                      0.05  0.844478   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.073154   
                                                      0.96  0.073154   
                                                      0.97  0.073154   
                                                      0.98  0.073154   
                                                      0.99  0.073154   

                                                                   f  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.040723   
                                                      0.02  0.040723   
                                                      0.03  0.040723   
                                                      0.04  0.040726   
                                                      0.05  0.040747   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.124518   
                                                      0.96  0.124518   
                                                      0.97  0.124518   
                                                      0.98  0.124518   
                                                      0.99  0.124518   

                                                                 wpr  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.014994   
                                                      0.02  0.014994   
                             

In [76]:
# Identify best methods and thresholds
index_best = df.groupby(level=['group', 'ns'])[metric].idxmax() if metric in ['f', 'wf'] else df.groupby(['group', 'ns'])[metric].idxmin()
index_best

group             ns                
DeepGO            biological_process         (DeepGO, DeepGO_1, biological_process, 0.21)
                  cellular_component         (DeepGO, DeepGO_1, cellular_component, 0.13)
                  molecular_function         (DeepGO, DeepGO_2, molecular_function, 0.21)
INGA              biological_process             (INGA, INGA_2, biological_process, 0.41)
                  cellular_component             (INGA, INGA_2, cellular_component, 0.52)
                                                              ...                        
tianlab1          cellular_component       (tianlab1, tianlab1, cellular_component, 0.36)
                  molecular_function       (tianlab1, tianlab1, molecular_function, 0.35)
zhang_freddolino  biological_process    (zhang_freddolino, zhang_freddolino_1, biologi...
                  cellular_component    (zhang_freddolino, zhang_freddolino_2, cellula...
                  molecular_function    (zhang_freddolino, zhan

In [77]:
# Filter the dataframe for the best method and threshold
df_best = df.loc[index_best, ['cov', 'colors'] + cols + [metric]]
df_best

cov  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.21  1.000000   
                                    cellular_component 0.13  1.000000   
                 DeepGO_2           molecular_function 0.21  1.000000   
INGA             INGA_2             biological_process 0.41  0.914894   
                                    cellular_component 0.52  1.000000   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.36  1.000000   
                                    molecular_function 0.35  0.849224   
zhang_freddolino zhang_freddolino_1 biological_process 0.47  1.000000   
                 zhang_freddolino_2 cellular_component 0.27  1.000000   
                 zhang_freddolino_1 molecular_function 0.41  1.000000   

                                                                                                        colors  \
group            label              ns                 tau                                                       
DeepGO           DeepGO_1           biological_process 0.21  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    cellular_component 0.13  (0.09019607843137255, 0.7450980392156863, 0.81...   
                 DeepGO_2           molecular_function 0.21  (0.09019607843137255, 0.7450980392156863, 0.81...   
INGA             INGA_2             biological_process 0.41  (0.8392156862745098, 0.15294117647058825, 0.15...   
                                    cellular_component 0.52  (0.8392156862745098, 0.15294117647058825, 0.15...   
...                                                                                                        ...   
tianlab1         tianlab1           cellular_component 0.36  (0.7372549019607844, 0.7411764705882353, 0.133...   
                                    molecular_function 0.35  (0.7372549019607844, 0.7411764705882353, 0.133...   
zhang_freddolino zhang_freddolino_1 biological_process 0.47  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_2 cellular_component 0.27  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_1 molecular_function 0.41  (0.596078431372549, 0.8745098039215686, 0.5411...   

                                                                  wrc  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.21  0.090133   
                                    cellular_component 0.13  0.158849   
                 DeepGO_2           molecular_function 0.21  0.177822   
INGA             INGA_2             biological_process 0.41  0.287657   
                                    cellular_component 0.52  0.481346   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.36  0.430724   
                                    molecular_function 0.35  0.373223   
zhang_freddolino zhang_freddolino_1 biological_process 0.47  0.158423   
                 zhang_freddolino_2 cellular_component 0.27  0.447324   
                 zhang_freddolino_1 molecular_function 0.41  0.316020   

                                                                  wpr  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.21  0.141876   
                                    cellular_component 0.13  0.424807   
                 DeepGO_2           molecular_function 0.21  0.055043   
INGA             INGA_2             biological_process 0.41  0.321581   
                                    cellular_component 0.52  0.322923   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.36  0.442188   
                          

In [78]:
# Filter the dataframe for the best methods
df_methods = df.reset_index('tau').loc[[ele[:-1] for ele in index_best], ['tau', 'cov', 'colors'] + cols + [metric]].sort_index()
# Makes the curves monotonic. Cumulative max on the last column of the cols variable, e.g. "pr" --> precision
if metric in ['f', 'wf']:
    df_methods[cols[-1]] = df_methods.groupby(level=['label', 'ns'])[cols[-1]].cummax()
else:
    df_methods[cols[-1]] = df_methods.groupby(level=['label', 'ns'])[cols[-1]].cummin()
# Save to file
df_methods.drop(columns=['colors']).to_csv('{}/eval_{}.tsv'.format(out_folder, metric), float_format="%.3f", sep="\t")
df_methods

tau       cov  \
group            label              ns                                   
DeepGO           DeepGO_1           biological_process  0.01  1.000000   
                                    biological_process  0.02  1.000000   
                                    biological_process  0.03  1.000000   
                                    biological_process  0.04  1.000000   
                                    biological_process  0.05  1.000000   
...                                                      ...       ...   
zhang_freddolino zhang_freddolino_2 cellular_component  0.95  0.935737   
                                    cellular_component  0.96  0.931034   
                                    cellular_component  0.97  0.931034   
                                    cellular_component  0.98  0.929467   
                                    cellular_component  0.99  0.929467   

                                                                                                   colors  \
group            label              ns                                                                      
DeepGO           DeepGO_1           biological_process  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    biological_process  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    biological_process  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    biological_process  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    biological_process  (0.09019607843137255, 0.7450980392156863, 0.81...   
...                                                                                                   ...   
zhang_freddolino zhang_freddolino_2 cellular_component  (0.596078431372549, 0.8745098039215686, 0.5411...   
                                    cellular_component  (0.596078431372549, 0.8745098039215686, 0.5411...   
                                    cellular_component  (0.596078431372549, 0.8745098039215686, 0.5411...   
                                    cellular_component  (0.596078431372549, 0.8745098039215686, 0.5411...   
                                    cellular_component  (0.596078431372549, 0.8745098039215686, 0.5411...   

                                                             wrc       wpr  \
group            label              ns                                       
DeepGO           DeepGO_1           biological_process  0.163733  0.061316   
                                    biological_process  0.163733  0.061316   
                                    biological_process  0.163733  0.061316   
                                    biological_process  0.163733  0.061316   
                                    biological_process  0.163733  0.061316   
...                                                          ...       ...   
zhang_freddolino zhang_freddolino_2 cellular_component  0.078014  0.549380   
                                    cellular_component  0.074414  0.556740   
                                    cellular_component  0.074414  0.556740   
                                    cellular_component  0.070145  0.564111   
                                    cellular_component  0.067612  0.573499   

                                                              wf  
group            label              ns                            
DeepGO           DeepGO_1           biological_process  0.089220  
                                    biological_process  0.089220  
                                    biological_process  0.089220  
                                    biological_process  0.089220  
                                    biological_process  0.089220  
...                                                          ...  
zhang_freddolino zhang_freddolino_2 cellular_component  0.136626  
                                    cellular_component  0.

In [79]:
# Calculate the max coverage across all thresholds
df_best['max_cov'] = df_methods.groupby(level=['group', 'label', 'ns'])['cov'].max()
df_best

cov  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.21  1.000000   
                                    cellular_component 0.13  1.000000   
                 DeepGO_2           molecular_function 0.21  1.000000   
INGA             INGA_2             biological_process 0.41  0.914894   
                                    cellular_component 0.52  1.000000   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.36  1.000000   
                                    molecular_function 0.35  0.849224   
zhang_freddolino zhang_freddolino_1 biological_process 0.47  1.000000   
                 zhang_freddolino_2 cellular_component 0.27  1.000000   
                 zhang_freddolino_1 molecular_function 0.41  1.000000   

                                                                                                        colors  \
group            label              ns                 tau                                                       
DeepGO           DeepGO_1           biological_process 0.21  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    cellular_component 0.13  (0.09019607843137255, 0.7450980392156863, 0.81...   
                 DeepGO_2           molecular_function 0.21  (0.09019607843137255, 0.7450980392156863, 0.81...   
INGA             INGA_2             biological_process 0.41  (0.8392156862745098, 0.15294117647058825, 0.15...   
                                    cellular_component 0.52  (0.8392156862745098, 0.15294117647058825, 0.15...   
...                                                                                                        ...   
tianlab1         tianlab1           cellular_component 0.36  (0.7372549019607844, 0.7411764705882353, 0.133...   
                                    molecular_function 0.35  (0.7372549019607844, 0.7411764705882353, 0.133...   
zhang_freddolino zhang_freddolino_1 biological_process 0.47  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_2 cellular_component 0.27  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_1 molecular_function 0.41  (0.596078431372549, 0.8745098039215686, 0.5411...   

                                                                  wrc  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.21  0.090133   
                                    cellular_component 0.13  0.158849   
                 DeepGO_2           molecular_function 0.21  0.177822   
INGA             INGA_2             biological_process 0.41  0.287657   
                                    cellular_component 0.52  0.481346   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.36  0.430724   
                                    molecular_function 0.35  0.373223   
zhang_freddolino zhang_freddolino_1 biological_process 0.47  0.158423   
                 zhang_freddolino_2 cellular_component 0.27  0.447324   
                 zhang_freddolino_1 molecular_function 0.41  0.316020   

                                                                  wpr  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.21  0.141876   
                                    cellular_component 0.13  0.424807   
                 DeepGO_2           molecular_function 0.21  0.055043   
INGA             INGA_2             biological_process 0.41  0.321581   
                                    cellular_component 0.52  0.322923   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.36  0.442188   
                          

In [80]:
# Set a label column for the plot legend
df_best['label'] = df_best.index.get_level_values('label')
df_best['label'] = df_best.agg(lambda x: f"{x['label']} ({metric.upper()}={x[metric]:.3f} C={x['max_cov']:.3f})", axis=1)
df_best

cov  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.21  1.000000   
                                    cellular_component 0.13  1.000000   
                 DeepGO_2           molecular_function 0.21  1.000000   
INGA             INGA_2             biological_process 0.41  0.914894   
                                    cellular_component 0.52  1.000000   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.36  1.000000   
                                    molecular_function 0.35  0.849224   
zhang_freddolino zhang_freddolino_1 biological_process 0.47  1.000000   
                 zhang_freddolino_2 cellular_component 0.27  1.000000   
                 zhang_freddolino_1 molecular_function 0.41  1.000000   

                                                                                                        colors  \
group            label              ns                 tau                                                       
DeepGO           DeepGO_1           biological_process 0.21  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    cellular_component 0.13  (0.09019607843137255, 0.7450980392156863, 0.81...   
                 DeepGO_2           molecular_function 0.21  (0.09019607843137255, 0.7450980392156863, 0.81...   
INGA             INGA_2             biological_process 0.41  (0.8392156862745098, 0.15294117647058825, 0.15...   
                                    cellular_component 0.52  (0.8392156862745098, 0.15294117647058825, 0.15...   
...                                                                                                        ...   
tianlab1         tianlab1           cellular_component 0.36  (0.7372549019607844, 0.7411764705882353, 0.133...   
                                    molecular_function 0.35  (0.7372549019607844, 0.7411764705882353, 0.133...   
zhang_freddolino zhang_freddolino_1 biological_process 0.47  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_2 cellular_component 0.27  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_1 molecular_function 0.41  (0.596078431372549, 0.8745098039215686, 0.5411...   

                                                                  wrc  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.21  0.090133   
                                    cellular_component 0.13  0.158849   
                 DeepGO_2           molecular_function 0.21  0.177822   
INGA             INGA_2             biological_process 0.41  0.287657   
                                    cellular_component 0.52  0.481346   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.36  0.430724   
                                    molecular_function 0.35  0.373223   
zhang_freddolino zhang_freddolino_1 biological_process 0.47  0.158423   
                 zhang_freddolino_2 cellular_component 0.27  0.447324   
                 zhang_freddolino_1 molecular_function 0.41  0.316020   

                                                                  wpr  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.21  0.141876   
                                    cellular_component 0.13  0.424807   
                 DeepGO_2           molecular_function 0.21  0.055043   
INGA             INGA_2             biological_process 0.41  0.321581   
                                    cellular_component 0.52  0.322923   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.36  0.442188   
                          

In [81]:
plt.rcParams.update({'font.size': 22})
for ns, df_g in df_best.groupby(level='ns'):
    fig, ax = plt.subplots(figsize=(15, 15))
    for i, (index, row) in enumerate(df_g.sort_values(by=[metric, 'max_cov'], ascending=[False if metric in ['f', 'wf'] else True, False]).iterrows()):
        # print(row)
        data = df_methods.loc[index[:-1]]
        ax.plot(data[cols[0]], data[cols[1]], color=row['colors'], label=row['label'], lw=2, zorder=500-i)
        ax.plot(row[cols[0]], row[cols[1]], color=row['colors'], marker='o', markersize=12, mfc='none', zorder=1000-i)
        ax.plot(row[cols[0]], row[cols[1]], color=row['colors'], marker='o', markersize=6, zorder=1000-i)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    plt.xlim(0, max(1, df_best.loc[:,:,ns,:][cols[0]].max()))
    plt.ylim(0, max(1, df_best.loc[:,:,ns,:][cols[1]].max()))

    ax.set_title(ns)
    ax.set_xlabel(cols[0])
    ax.set_ylabel(cols[1])

    plt.savefig("{}/fig_{}_{}.png".format(out_folder, metric, ns), bbox_inches='tight')
    plt.clf()

<Figure size 1080x1080 with 0 Axes>

<Figure size 1080x1080 with 0 Axes>

<Figure size 1080x1080 with 0 Axes>